## Custom API Notebook

### Initialize the Node

In [1]:
# stdlib
from typing import Any

# syft absolute
import syft as sy
from syft import SyftError
from syft import SyftSuccess

node = sy.orchestra.launch(
    name="test-domain-1",
    dev_mode=True,
    create_producer=True,
    n_consumers=3,
    in_memory_workers=True,
    reset=True,
    port=8081,
)

domain_client = node.login(email="info@openmined.org", password="changethis")
domain_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Staging Protocol Changes...
Starting test-domain-1 server on 0.0.0.0:8081

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=3 image_uid=4757fe66c77047dbbec9dd44e59d7e46 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


INFO:     Started server process [221039]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8081 (Press CTRL+C to quit)


INFO:     127.0.0.1:38326 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:38340 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:38340 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Waiting for server to start Done.
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:38340 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:38340 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:38344 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

INFO:     127.0.0.1:38340 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:38346 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:38346 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:38346 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:38346 - "GET /api/v2/api?verify_key=6f311eb71196dbd88791974f8c8b2fd1751adc64d52ce2a17d8500787e54ead0&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:38356 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <user@openmined.org>
INFO:     127.0.0.1:58078 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:58094 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:58096 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:58106 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:58106 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:58106 - "POST /ap

### Create a public custom API Endpoint by using the decorator

This allows node admin to create a new public endpoint by using only the decorator.

In [2]:
@sy.api_endpoint(
    path="first.query",
    settings={"key": "value"},
)
def public_endpoint_method(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=public_endpoint_method)
response

SyftSuccess: Endpoint successfully created.

In [3]:
assert isinstance(response, SyftSuccess)

In [4]:
domain_client.api.services.api.api_endpoints()

[syft.service.api.api.TwinAPIEndpointView]

In [5]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

In [6]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <user@openmined.org>


In [7]:
assert domain_client.api.services.first.query(query="SELECT *")
assert domain_guest.api.services.first.query(query="SELECT *")

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [9]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def public_function(
    context,
) -> str:
    return "Public Function Execution"


@sy.api_endpoint_method(settings={"Hello": "Private"})
def private_function(
    context,
) -> str:
    return "Private Function Execution"


new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    mock_function=public_function,
    private_function=private_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)

In [10]:
domain_client.api.services.api.api_endpoints()

[syft.service.api.api.TwinAPIEndpointView,
 syft.service.api.api.TwinAPIEndpointView]

In [11]:
assert isinstance(response, SyftSuccess)
assert len(domain_client.api.services.api.api_endpoints()) == 2

In [12]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <user@openmined.org>


In [13]:
assert domain_client.api.services.third.query() == "Private Function Execution"

In [14]:
assert domain_guest.api.services.third.query() == "Public Function Execution"

In [15]:
domain_guest.api.services.third.query()

'Public Function Execution'

In [16]:
@sy.syft_function_single_use(
    endpoint=domain_guest.api.services.third.query,
)
def job_function(endpoint):
    return endpoint()


# Create a new project
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_guest],
)

result = new_project.create_code_request(job_function, domain_guest)
assert isinstance(result, SyftSuccess)

SyftSuccess: Syft function 'job_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [17]:
res = None
for r in domain_client.requests.get_all():
    if r.requesting_user_email == "user@openmined.org":
        res = r.approve()
assert res is not None, res
res

Approving request for domain test-domain-1


SyftSuccess: Request 39b9b846d37d43b0a1923c45c3e3b4c1 changes applied

In [18]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

Pointer:
None

In [19]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.

```python
Pointer
```
'Private Function Execution'

In [20]:
domain_client.api.services.third.query

## API: third.query
### Description: <span style="font-weight: normal;">Lore ipsulum ...</span><br>
#### Private Code:
```python
def private_function(
    context,
) -> str:
    return "Private Function Execution"

```
#### Public Code:
```python
def public_function(
    context,
) -> str:
    return "Public Function Execution"

```


In [21]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.

```python
Pointer
```
'Private Function Execution'

In [22]:
assert not isinstance(result, SyftError), result

In [23]:
assert result.get() == "Private Function Execution"

In [24]:
assert isinstance(domain_guest.api.services.third.query.private(), SyftError)

In [25]:
result = domain_client.api.services.api.delete(endpoint_path="third.query")
assert isinstance(result, SyftSuccess), result

In [26]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

## Updating Endpoints

First we'll create a new endpoint

In [27]:
@sy.api_endpoint(
    path="test.update",
    settings={"key": "value"},
)
def new_public_function(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_public_function)
assert isinstance(response, SyftSuccess), response
response

SyftSuccess: Endpoint successfully created.

#### Update the public function

In [29]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def updated_public_function(
    context,
) -> str:
    return "Updated Public Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", mock_function=updated_public_function
)
assert isinstance(response, SyftSuccess), response
response

SyftSuccess: Endpoint successfully updated.

#### Update the private function

In [30]:
@sy.api_endpoint_method(settings={"Hello": "Private"})
def updated_private_function(
    context,
) -> str:
    return "Updated Private Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", private_function=updated_private_function
)
assert isinstance(response, SyftSuccess), response
response

SyftSuccess: Endpoint successfully updated.

#### Update both functions with a pair that has a new signature

In [31]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def new_sig_public_function(context, new_parameter) -> str:
    return "Updated Public Function Execution"


@sy.api_endpoint_method(settings={"Hello": "Private"})
def new_sig_private_function(context, new_parameter) -> str:
    return "Updated Private Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update",
    mock_function=new_sig_public_function,
    private_function=new_sig_private_function,
)
assert isinstance(response, SyftSuccess), response
response

SyftSuccess: Endpoint successfully updated.

### Invalid update attempts
- Both functions empty
- Signature mismatch
- Non existing endpoint

#### Both functions are empty

In [32]:
response = domain_client.api.services.api.update(endpoint_path="test.update")
assert isinstance(response, SyftError), response

#### Signature mismatch

In [33]:
@sy.api_endpoint_method(settings={"Hello": "Public"})
def bad_public_function(context, foo) -> str:
    return "Updated Public Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", mock_function=bad_public_function
)
assert isinstance(response, SyftError), response

Non Existing endpoint

In [34]:
response = domain_client.api.services.api.update(
    endpoint_path="nonexistent", mock_function=bad_public_function
)
assert isinstance(response, SyftError), response